In [5]:
import os
import cv2
import glob
import random
import numpy as np
import imgaug as ia
import pandas as pd
import tensorflow as tf
from progressbar import * #进度条
from natsort import natsorted
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
from imgaug import parameters as iap
from skimage.transform import resize
from skimage.io import imread, imsave
from sklearn.metrics import confusion_matrix
from skimage import transform, filters, exposure

os.environ['CUDA_VISIBLE_DEVICES']='1'
%matplotlib inline

In [2]:
#由于数据集较大，采用tensorflow自带的TFRecord进行读取
def Trainset2TFRecord2(train_tfrecords_path, val_tfrecords_path, ratio=0.9, data_path='data'):
    files = natsorted(glob.glob(data_path + '/train/*'))
    total = len(files)
    train_num = int(total*ratio)
    val_num = total-train_num
    if (os.path.exists(train_tfrecords_path) and os.path.exists(val_tfrecords_path)):
        print('data have already processed')
        return train_num, val_num
    
    writer_train = tf.python_io.TFRecordWriter(train_tfrecords_path)
    writer_val = tf.python_io.TFRecordWriter(val_tfrecords_path)
    
    labels = pd.read_csv(data_path + '/trainLabels.csv')
    
    pbar = ProgressBar().start()
    for i, file_path in enumerate(files):
        pbar.update(int((i / (total - 1)) * 100))#进度条
        time.sleep(0.01)

        img_raw = tf.gfile.FastGFile(file_path, 'rb').read()
        label = label2int[labels.loc[i].label]

        example = tf.train.Example(features=tf.train.Features(feature={
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
        }))
        if i < train_num:
            writer_train.write(example.SerializeToString())
        else:
            writer_val.write(example.SerializeToString())
    writer_train.close()
    writer_val.close()
    pbar.finish()
    
    return train_num, val_num
def TFRecord2TrainData2(train_tfrecords_path, num_class, batch_size, num_epochs, shuffle=True):
    filename_queue = tf.train.string_input_producer([train_tfrecords_path], num_epochs=num_epochs)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                   features={
                                       'img_raw' : tf.FixedLenFeature([], tf.string),
                                       'label': tf.FixedLenFeature([], tf.int64)
                                   })
    img = tf.image.decode_png(features['img_raw'])  # 与方式一的不同点在于需要用decode_png/decode_jpeg解码
                                                    # output an RGB image. [height, width, channels]
    img = tf.reshape(img, [32, 32, 3])
    img = tf.image.resize_images(img, [224, 224])
    label = tf.cast(features['label'], tf.int32)
    label = tf.one_hot(label,num_class,1,0)
    if shuffle==True:
        img_batch, label_batch = tf.train.shuffle_batch([img, label], batch_size=batch_size,\
                                                        capacity=500 + 3 * batch_size, min_after_dequeue=500)
    else:
        img_batch, label_batch = tf.train.batch([img, label], batch_size=batch_size, capacity= 3 * batch_size)
    return img_batch, label_batch

In [3]:
train_tfrecords_path = 'data/train.tfrecord'
val_tfrecords_path = 'data/val.tfrecord'
train_num, val_num = Trainset2TFRecord2(train_tfrecords_path, val_tfrecords_path)
print(train_num, val_num)

data have already processed
45000 5000


In [4]:
#解码并查看效果
#img,label = TFRecord2TrainData(train_tfrecords_path, 10)
#img_batch, label_batch = tf.train.shuffle_batch([img, label], batch_size=500, capacity=50000, min_after_dequeue=1000, num_threads=1)
#capacity是队列的最大容量, min_after_dequeue是dequeue后最小的队列大小,这个代表队列中的元素大于它的时候就输出乱的顺序的batch
#num_threads是进行队列操作的线程数。
img_train_batch, label_train_batch = TFRecord2TrainData2(train_tfrecords_path, 10, 128, 5)
img_val_batch, label_val_batch = TFRecord2TrainData2(val_tfrecords_path, 10, 128, 5)#10类 batch_size=128, epoch = 5
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    #先产生！个看下效果
    '''
    for epoch in range(10):
        try:
            while not coord.should_stop():
                img_, label_ = sess.run([img_batch, label_batch]) 
                print(epoch)
        except tf.errors.OutOfRangeError:
            print("done")
        finally:
            coord.request_stop()
        coord.join(threads)
    ''' 
    #验证集上测试
    try:
        train_total_batch = int(train_num/128)
        val_total_bath = int(val_num/128)
        epoch = 0
        while not coord.should_stop():#使用 coord.should_stop()来查询是否应该终止所有线程
            #开始一个epoch的训练
            train_begin = 0
            val_begin = 0
            for i in range(train_total_batch):
                img_, label_ = sess.run([img_train_batch, label_train_batch]) 
                if train_begin == 0:
                    print("train")
                    print(epoch, img_.shape)
                    train_begin = 1
            for i in range(val_total_bath):
                img_, label_ = sess.run([img_val_batch, label_val_batch]) 
                if val_begin == 0:
                    print("val")
                    print(epoch, img_.shape)
                    val_begin = 1
            epoch+=1
    except tf.errors.OutOfRangeError:
        print("done")
    finally:       
        coord.request_stop()#使用coord.request_stop()来发出终止所有线程的命令
        coord.join(threads)#使用coord.join(threads)把线程加入主线程，等待threads结束。
    '''
    for i in range(10): #产生10个batch
        img_, label_ = sess.run([img_batch, label_batch])#每次输出queue中的一个batch
        plt.subplot(2, 5, i+1)
        label = int2lable[np.argmax(label_, axis=1)[0]]
        plt.title(label)
        plt.imshow(img_[0].astype('uint8'))
        i += 1
    plt.show()
    coord.request_stop()
    coord.join(threads)
    '''

train
0 (128, 224, 224, 3)
val
0 (128, 224, 224, 3)
train
1 (128, 224, 224, 3)
val
1 (128, 224, 224, 3)
train
2 (128, 224, 224, 3)
val
2 (128, 224, 224, 3)
train
3 (128, 224, 224, 3)
val
3 (128, 224, 224, 3)
train
4 (128, 224, 224, 3)
val
4 (128, 224, 224, 3)
train
5 (128, 224, 224, 3)
done


In [ ]:
def batch_augment(data):
    seq = iaa.Sequential([
        iaa.Crop(percent=0.01), # # 从每侧裁剪图像0到16px（随机选择）
        iaa.Fliplr(0.5), # 水平翻转图像 括号内为Probability of each image to get flipped.
        iaa.Flipud(0.5), #上下翻转
        #iaa.GaussianBlur(sigma=(0, 3.0)),  # 使用0到3.0的sigma模糊图像
        iaa.Affine(scale=(0.7, 1.3), translate_percent=0.01, rotate=iap.Normal(-20, 20)),#旋转
        iaa.Multiply(iap.Positive(iap.Normal(0.0, 0.1)) + 1.0),#明暗变化
        #iaa.AddElementwise(iap.Discretize((iap.Beta(0.5, 0.5) * 2 - 1.0) * 64))
        #iaa.AdditiveGaussianNoise(scale=(0,  0.05*255)),
        #iaa.Sharpen(alpha=0.5),
        #iaa.Scale((0.5, 1.5))
    ],random_order=True)#每个batch中的Augmenters顺序不一样
    x_batch = seq.augment_images(data)
    return x_batch

In [ ]:
batch_size = 5
img_train_batch, label_train_batch = TFRecord2TrainData2(train_tfrecords_path, 10, batch_size, 1)
img_val_batch, label_val_batch = TFRecord2TrainData2(val_tfrecords_path, 10, batch_size, 1)#10类 batch_size=128, epoch = 1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(1) : #产生1个batch
        img_batch, label_batch = sess.run([img_train_batch, label_train_batch])#每次输出queue中的一个batch
        for j in range(batch_size):
            plt.subplot(2, batch_size, j+1)
            label = int2lable[np.argmax(label_batch, axis=1)[j]]
            plt.title(label)
            plt.imshow(img_batch[j].astype('uint8'))
        #扩增后的数据
        '''
        for j in range(batch_size):
            plt.subplot(2, batch_size, j+batch_size+1)
            label = int2lable[np.argmax(label_batch, axis=1)[j]]
            plt.title(label)
            plt.imshow(img_batch[j].astype('uint8'))
        '''
        img_aug = batch_augment(img_batch)
        for j in range(batch_size):
            plt.subplot(2, batch_size, j+batch_size+1)
            label = int2lable[np.argmax(label_batch, axis=1)[j]]
            plt.title(label)
            plt.imshow(img_aug[j].astype('uint8'))
        
    plt.show()
    coord.request_stop()
    coord.join(threads)

In [ ]:
### 查看预训练模型中的参数 ###
ckpt_path = './model/vgg19/mode.ckpt'
pre_model_path = 'model/vgg19/vgg19.npy'
pre_model = np.load(pre_model_path, encoding = "bytes").item()
layers = list(pre_model.keys())#将dict转换为list  层的名称
#layer_1 = pre_model[layers[0]] #第一层中内容
#layer_1_w =  pre_model[layers[0]][0]  #第一层的权重
print(layers)
layer_1_0 =  pre_model[layers[0]][1] #第一层的偏差
print(pre_model[layers[0]][0].shape) #第一层权重的形状 (3, 3, 3, 64)
print(pre_model[layers[0]][1].shape) #第一层的偏差的形状 (64,)